In [20]:
import numpy as np

def viterbi_algorithm(S, Pi, E, P, Y):
    K = len(S) # Length of state space will give K
    T = len(Y) # Length of observation sequence will give T
    
    # Initialize matrices for Viterbi probabilities and paths
    viterbi_prob = np.zeros((K, T))
    viterbi_path = np.zeros((K, T))
    
    # Part I: Initialization
    for i in range(K):
        viterbi_prob[i][0] = Pi[i] * E[i][Y[0]]
        viterbi_path[i][0] = 0
    
    # Part II: Compute Viterbi probabilities and path
    for j in range(1, T):
        for i in range(K):
            viterbi_prob[i, j] = max(E[i][Y[j]] * P[k][i] * viterbi_prob[k, j-1] for k in range(K))
            viterbi_path[i, j] = np.argmax([E[i][Y[j]] * P[k][i] * viterbi_prob[k, j-1] for k in range(K)])
    
    # Part III: Re-track the most likely path
    x = []
    x_t = np.argmax(viterbi_prob[:, T-1])
    x.append(x_t)
    for j in range(T-1, 0, -1):
        x_prev = int(viterbi_path[int(x_t), j])
        x.append(x_prev)
        x_t = x_prev
        
    # Reverse path to get the correct order
    x.reverse()
    
    return x, np.max(viterbi_prob[:, T-1])

In [21]:
import numpy as np
def forward_algorithm(S, Pi, E, P, Y, Pii=0):
    K = len(S)
    T = len(Y)
    
    forward_prob = np.zeros((K, T))
    
    # Part I: Initialization
    for i in range(K):
        forward_prob[i][0] = Pi[i] * E[i][Y[0]]
    
    # Part II: Recursion
    for j in range(1, T):
        for i in range(K):
            forward_prob[i, j] = sum(forward_prob[k, j-1] * P[k][i] * E[i][Y[j]] for k in range(K))
    
    # Part III: Termination
    total_prob = np.sum(forward_prob[:, T-1])
    return total_prob


In [22]:
S = ['rainy', 'cloudy', 'sunny']
Pi = [0.35, 0.25, 0.4]
P = [
    [0.4, 0.2, 0.4],
    [0.2, 0.5, 0.3],
    [0.3, 0.6, 0.1]
]
E = [
    [0.8, 0.1, 0.1],
    [0.2, 0.5, 0.3],
    [0.4, 0.2, 0.4]
]
activities = ['read', 'read', 'shop', 'play', 'shop']
observation_indices = [0, 0, 2, 1, 2]  # Convert activities to observation indices(read->0,play->1,shop->2)

probability_activities = forward_algorithm(S, Pi, E, P, observation_indices)
print("Probability of the given sequence of activities:", probability_activities)

Probability of the given sequence of activities: 0.0054941706


In [23]:
# Given data
S = ['normal', 'alternate', 'direct']
Pi = [0.2, 0.3, 0.5]
P = [
    [0.35, 0.45, 0.2],
    [0.28, 0.32, 0.4],
    [0.75, 0.1, 0.15]
]
E = [
    [0.1, 0.9],
    [0.6, 0.4],
    [0.7, 0.3]
]
#pitch_data = ['up', 'down', 'down', 'down', 'up', 'up', 'up', 'down', 'up', 'down']
pitch_data = ['up', 'down']

# Convert pitch data to observation indices
observation_indices = [0 if pitch == 'up' else 1 for pitch in pitch_data]

# (b) Calculate the probability of the given pitch data sequences
Y_sequences = [
    ['up'] * 8,
    ['down', 'up', 'down', 'up', 'down', 'up', 'down', 'up'],
    ['up', 'down'] * 4,
    ['up', 'up', 'down', 'up', 'down', 'down', 'down', 'up'],
    ['down'] * 8
]

for i, Y in enumerate(Y_sequences, start=1):
    observation_indices = [0 if pitch == 'up' else 1 for pitch in Y]
    probability = forward_algorithm(S, Pi, E, P, observation_indices)
    print(f"(b){i}. Probability of {Y}: {probability}")

(b)1. Probability of ['up', 'up', 'up', 'up', 'up', 'up', 'up', 'up']: 0.00044796686548259797
(b)2. Probability of ['down', 'up', 'down', 'up', 'down', 'up', 'down', 'up']: 0.00302268125892117
(b)3. Probability of ['up', 'down', 'up', 'down', 'up', 'down', 'up', 'down']: 0.006147664980957777
(b)4. Probability of ['up', 'up', 'down', 'up', 'down', 'down', 'down', 'up']: 0.004227104015319532
(b)5. Probability of ['down', 'down', 'down', 'down', 'down', 'down', 'down', 'down']: 0.009115099658980729


In [24]:
#2(c)(i)
import numpy as np

def adjust_emission_probabilities(E):
    # Assign equal probabilities to 'up' and 'down' observations for each state
    num_states, num_observations = np.shape(E)
    equal_probability = 1 / num_observations
    adjusted_E = np.full((num_states, num_observations), equal_probability)
    return adjusted_E

# Given data
S = ['normal', 'alternate', 'direct']
Pi = [0.2, 0.3, 0.5]
P = [
    [0.35, 0.45, 0.2],
    [0.28, 0.32, 0.4],
    [0.75, 0.1, 0.15]
]
E = [
    [0.1, 0.9],
    [0.6, 0.4],
    [0.7, 0.3]
]
pitch_data = ['up', 'down', 'down', 'down', 'up', 'up', 'up', 'down', 'up', 'down']

# Adjust emission probabilities
adjusted_E = adjust_emission_probabilities(E)

# Convert pitch data to observation indices
observation_indices = [0 if pitch == 'up' else 1 for pitch in pitch_data]

# Implement Viterbi algorithm with adjusted parameters
most_likely_control_sequence, _ = viterbi_algorithm(S, Pi, adjusted_E, P, observation_indices)
print("Most likely sequence of aircraft control laws with adjusted emission probabilities:", [S[state] for state in most_likely_control_sequence])



Most likely sequence of aircraft control laws with adjusted emission probabilities: ['direct', 'normal', 'alternate', 'direct', 'normal', 'alternate', 'direct', 'normal', 'alternate', 'direct']


In [25]:
#2(c)(ii)
import numpy as np

def adjust_emission_probabilities(E):
    # Assign equal probabilities to 'up' and 'down' observations for each state
    num_states, num_observations = np.shape(E)
    equal_probability = 1 / num_observations
    adjusted_E = np.full((num_states, num_observations), equal_probability)
    return adjusted_E

def adjust_initial_probabilities(Pi):
    # Assign equal initial probabilities to all states
    num_states = len(Pi)
    equal_probability = 1 / num_states
    adjusted_Pi = np.full(num_states, equal_probability)
    return adjusted_Pi

# Given data
S = ['normal', 'alternate', 'direct']
Pi = [0.2, 0.3, 0.5]
P = [
    [0.35, 0.45, 0.2],
    [0.28, 0.32, 0.4],
    [0.75, 0.1, 0.15]
]
E = [
    [0.1, 0.9],
    [0.6, 0.4],
    [0.7, 0.3]
]
pitch_data = ['up', 'down', 'down', 'down', 'up', 'up', 'up', 'down', 'up', 'down']

# Adjust emission probabilities
adjusted_E = adjust_emission_probabilities(E)

# Adjust initial probabilities
adjusted_Pi = adjust_initial_probabilities(Pi)

# Convert pitch data to observation indices
observation_indices = [0 if pitch == 'up' else 1 for pitch in pitch_data]

# Implement Viterbi algorithm with adjusted parameters
most_likely_control_sequence, _ = viterbi_algorithm(S, adjusted_Pi, adjusted_E, P, observation_indices)
print("Most likely sequence of aircraft control laws with adjusted parameters:", [S[state] for state in most_likely_control_sequence])


Most likely sequence of aircraft control laws with adjusted parameters: ['normal', 'alternate', 'direct', 'normal', 'alternate', 'direct', 'normal', 'alternate', 'direct', 'normal']


In [26]:
# Given emission matrix E
E = np.array([
    [0.65, 0.35],
    [0.82, 0.18],
    [0.73, 0.27]
])

# Given initial probability distribution Π
Pi = np.array([0.2, 0.3, 0.5])

# Given probability transition matrix P
P = np.array([
    [0.35, 0.45, 0.2],
    [0.28, 0.32, 0.4],
    [0.75, 0.1, 0.15]
])

# Define the states
S = ['normal', 'alternate', 'direct']

# (a) (i)
actions = ['landing', 'take-off', 'cruising horizontally']
for i, action in enumerate(actions):
    probabilities = E[i]  # Probabilities of observations for the current action
    observation = 'pitch up' if probabilities[0] > probabilities[1] else 'pitch down'
    print(f"For {action}, the aircraft is most likely performing {observation}")

# (a) (ii)
E_tilde = E[:, ::-1]  # Swap the columns of E
print("Swapped emission matrix E~:\n", E_tilde)
# If the probability of observing pitch up decreases significantly and pitch down increases,
# it might 0indicate potential anomalies during take-off, such as instability or unexpected behavior.

# (b) Algorithm to compute the probability of the company receiving the given pitch data
# in the next eight 5-minute intervals, starting from a specific time instant t.
def compute_probability_next_intervals(initial_state, pitch_data):
    # Convert pitch data to observation indices
    observations = [0 if pitch == 'up' else 1 for pitch in pitch_data]
    # Compute the probability using the forward algorithm
    probability = forward_algorithm(S, Pi, E, P, observations, initial_state)
    return probability

initial_state = 'alternate'  # Given Markovian state at time t
pitch_data = ['down', 'down', 'down', 'up', 'up', 'up', 'down', 'up']
probability_next_intervals = compute_probability_next_intervals(initial_state, pitch_data)
print("Probability of receiving the given pitch data in the next eight 5-minute intervals:", probability_next_intervals)


For landing, the aircraft is most likely performing pitch up
For take-off, the aircraft is most likely performing pitch up
For cruising horizontally, the aircraft is most likely performing pitch up
Swapped emission matrix E~:
 [[0.35 0.65]
 [0.18 0.82]
 [0.27 0.73]]
Probability of receiving the given pitch data in the next eight 5-minute intervals: 0.001565406338323729


In [28]:
# Given data
S = ['normal', 'alternate']
Pi = [0.5, 0.5]
P = [
    [0.5, 0.5],
    [0.5, 0.5]
]
E = [
    [0.5, 0.5],
    [0.5, 0.5]
]
pitch_data = ['up', 'down']
observation_indices = [0, 1]

# (b) Calculate the probability of the given pitch data sequences
Y_sequences = [
    ['up'] * 2
]

for i, Y in enumerate(Y_sequences, start=1):
    observation_indices = [0 if pitch == 'up' else 1 for pitch in Y]
    probability = forward_algorithm(S, Pi, E, P, observation_indices)
    print(f"(b){i}. Probability of {Y}: {probability}")

(b)1. Probability of ['up', 'up']: 0.25
